In [41]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
from geopy import geocoders  
from geopy.geocoders import Nominatim
from datetime import datetime
from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy.time import Time

In [31]:
def julian_date(date_time):
    jd = date_time.toordinal() + 1721424.5 + (date_time.hour + date_time.minute / 60 + date_time.second / 3600) / 24
    return jd

In [11]:
def gst_from_jd(jd):
    '''
    Calculate Greenwich Sidereal Time (GST) from Julian Date.
    '''
    T = (jd - 2451545.0) / 36525
    GST = 280.46061837 + 360.98564736629 * (jd - 2451545) + T**2 * (0.000387933 - T / 38710000)
    GST = GST % 360 # Normalize to 0 - 360 degrees
    return GST

In [12]:
def lst_from_gst(gst, longitude):
    """
    Calculate Local Sidereal Time (LST) from Greenwich Sidereal Time (GST) and longitude.
    """
    LST = gst + longitude
    LST = LST % 360  # Normalize to 0 - 360 degrees
    return LST

In [13]:
def zenith_ra_dec(latitude, longitude, date_time):
    """
    Compute the Right Ascension (RA) and Declination (Dec) of the zenith for a given location and date.
    """
    jd = julian_date(date_time)
    gst = gst_from_jd(jd)
    lst = lst_from_gst(gst, longitude)
    dec = latitude
    ra = lst
    return ra, dec

In [64]:
def map_eccentricity_to_trait(eccentricity):
    if eccentricity == 0:
        return "are perfectly stable"
    elif 0 < eccentricity < 0.3:
        return "prefer stability"
    elif 0.3 <= eccentricity < 0.6:
        return "are balanced"
    elif 0.6 <= eccentricity < 0.9:
        return "prefer excitement"
    else:
        return "embrace change"

def map_semimajor_axis_to_trait(semimajor_axis):
    if semimajor_axis < 0.1:
        return "are 'close to the action' and constantly influenced by your star's energy, suggesting a very outgoing and active nature"
    elif 0.1 <= semimajor_axis < 1:
        return "are still within a region of significant stellar influence, indicating a generally social and engaging character"
    elif 1 <= semimajor_axis < 5:
        return "strike a balance between the inner and outer regions, reflecting a well-rounded personality that is equally comfortable in social situations and solitude"
    elif 5 <= semimajor_axis < 30:
        return "are farther from the star, implying a more reserved and introspective nature, preferring less direct interaction"
    else:
        return "are on the outskirts, indicating a highly introspective and solitary disposition, thriving in their own space away from the hustle and bustle"

In [65]:
def map_orbital_period_to_trait(orbital_period):
    if orbital_period < 10:
        return "rapid orbits suggest a fast-paced and reactive thinking style"
    elif 10 <= orbital_period < 100:
        return "orbits allow for rapid changes and adaptation, indicating an active and adaptive thinking style"
    elif 100 <= orbital_period < 365:
        return "orbital periods tend to experience balanced conditions, suggesting a balanced and analytical thinking style"
    elif 365 <= orbital_period < 3650:
        return "planets take longer to orbit their stars, implying a more deliberate and thoughtful approach"
    else:
        return "very long orbital periods embody a reflective and contemplative thinking style"

def map_stellar_mass_to_trait(stellar_mass):
    if stellar_mass < 0.5:
        return "stable and enduring"
    elif 0.5 <= stellar_mass < 1.5:
        return "balanced and nurturing"
    elif 1.5 <= stellar_mass < 3:
        return "dynamic and charismatic"
    else:
        return "intense and transformative"

In [94]:
def generate_horoscope_message(user, year, month, day, hour, minute, second, citystate):
    geolocator = Nominatim(user_agent='moeur')
    location = geolocator.geocode(citystate)
    date_time = datetime(year, month, day, hour, minute, second)
    birth_ra, birth_dec = location[1][0], location[1][1]
    zen_ra, zen_dec = zenith_ra_dec(birth_ra, birth_dec, date_time)
    table = NasaExoplanetArchive.query_region(
    table="ps", coordinates=SkyCoord(zen_ra * u.deg, zen_dec * u.deg),
    radius=4 * u.deg) 
    coords = SkyCoord(zen_ra * u.deg, zen_dec * u.deg)
    stars_coords = SkyCoord(table['ra'], table['dec'], unit=(u.deg, u.deg))
    distances = coords.separation(stars_coords)
    closest_index = distances.argmin()
    closest_object = table[closest_index]
    closest_table = NasaExoplanetArchive.query_object(closest_object['pl_name'])
    name = closest_table['pl_name'][0]
    star = closest_table['hostname'][0]
    eccentricity = np.nanmean(closest_table["pl_orbeccen"])
    semi_major_axis = np.nanmean(np.asarray(closest_table["pl_orbsmax"].value))
    period = np.nanmean(np.asarray(closest_table["pl_orbper"].value))
    stellar_mass = np.nanmean(np.asarray(closest_table["st_mass"].value))
    
    eccentricity_trait = map_eccentricity_to_trait(eccentricity)
    axis_trait = map_semimajor_axis_to_trait(semi_major_axis)
    period_trait = map_orbital_period_to_trait(period)
    stellar_mass_trait = map_stellar_mass_to_trait(stellar_mass)
    
    message = (f"{user}, your birth exoplanet is {name} orbiting star {star}. "
               f"Based on an eccentricity of {eccentricity:.2f}, you {eccentricity_trait}. "
               f"With an orbit semi-major axis of {semi_major_axis:.2f} AU, you {axis_trait}. "
               f"With a birth exoplanet period of {period:.2f} days, these {period_trait}, "
               f"and with a stellar mass of {stellar_mass:.2f} solar masses, you are {stellar_mass_trait}.")
    
    return message

In [95]:
generate_horoscope_message('Chelsea', 1991, 4, 27, 12, 4, 0, "Redlands California")

"Chelsea, your birth exoplanet is WASP-3 b orbiting star WASP-3. Based on an eccentricity of 0.00, you prefer stability. With an orbit semi-major axis of 0.03 AU, you are 'close to the action' and constantly influenced by your star's energy, suggesting a very outgoing and active nature. With a birth exoplanet period of 1.85 days, these rapid orbits suggest a fast-paced and reactive thinking style, and with a stellar mass of 1.28 solar masses, you are balanced and nurturing."